# Mathema Grids

Comecemos por importar o módulo do Z3 bem como algumas ferramentas que nos serão úteis.

In [ ]:
from z3 import *
import operator

ops = {
    '+' : operator.add,
    '-' : operator.sub,
    '*' : operator.mul,
    '/' : operator.truediv
}

Em seguida, procedemos à obtenção do nome do ficheiro com o tabuleiro a resolver e criamos os dicionários necessários para guardar a informção contida no ficheiro.

In [ ]:
nomeFicheiro = input("Por favor insira o nome do ficheiro onde se encontra o tabuleiro.\n")
#abertura do ficheiro e leitura das linhas
ficheiro = open(nomeFicheiro, "r")
linhas = ficheiro.readlines()

#dicionario onde serão guardados os operadores das operações horizontais
opHorizontal = {}
#dicionario onde serão guardados os operadores das operações verticais
opVertical = {}
#dicionario onde serão guardados os resultados das operações horizontais
resHorizontal = {}
#dicionario onde serão guardados os resultados das operações verticais
resVertical = {}
#dicionario onde serão guardados as pistas, caso existam
valores = {}

Segue-se o armazenamento da informação nos dicionários.

In [ ]:
iLinha = 0

for linha in linhas:
    aux = linha.split()

    #a informação relativa às operações horizontais encontra-se nas linhas pares
    if (iLinha%2 == 0):
        for i in range(len(aux[0])):
            #nas posições pares da linha encontram-se as possíveis pistas
            if (i%2 == 0):
                if (int(iLinha/2)) not in valores:
                    valores[int(iLinha/2)] = {}

                if (aux[0][i] == '#'):
                    valores[int(iLinha/2)][int(i/2)] = 0
                else:
                    valores[int(iLinha/2)][int(i/2)] = int(aux[0][i])

            #nas posições ímpares da linha encontram-se os operadores
            else:
                if (int(iLinha/2)) not in opHorizontal:
                    opHorizontal[int(iLinha/2)] = {}

                opHorizontal[int(iLinha/2)][int(i/2)] = aux[0][i]

        #no fim da linha encontra-se o resultado pretendido na operação
        resHorizontal[int(iLinha/2)] = int(aux[1])
    
    #a informação relativa às operações verticais encontra-se nas linhas ímpares
    else:
        #na linha 5, que deverá ser a última linha do ficheiro, encontram-se os resultados
        #pretendidos para as operações verticais
        if (iLinha == 5):
            i = 0
            for res in aux:
                resVertical[i] = int(res)
                i += 1
        
        #nas restantes linhas encontram-se os operadores das operações verticais
        else:
            i = 0
            for res in aux:
                if i not in opVertical:
                    opVertical[i] = {}

                opVertical[i][int(iLinha/2)] = res
                i += 1

    iLinha += 1

ficheiro.close()

Após tudo isto, gera-se o Solver pretendido.

In [ ]:
s = Solver()

#dicionário para as variáveis do SMT Solver
x = {}

#criação das variáveis
for i in range(3):
    x[i] = {}
    for j in range(3):
        x[i][j] = Int('x' + str(i) + str(j))
        s.add(
            And(
                1 <= x[i][j],
                x[i][j] <= 9
            )
        )

        #caso haja uma pista para a posição correspondente à variável, guarda-se
        if(valores[i][j] != 0):
            s.add(x[i][j] == valores[i][j])

#não podem ocorrer valores repetidos
s.add(
    Distinct(
        [x[i][j] for i in range(3) for j in range(3)]
    )
)

#operações horizontais
for i in range(3):
    s.add(
        (ops[opHorizontal[i][1]](
            ops[opHorizontal[i][0]](
                x[i][0],
                x[i][1]
            ),
            x[i][2]
        ))
            ==
        resHorizontal[i]
    )

#operações verticais
for i in range(3):
    s.add(
        (ops[opVertical[i][1]](
            ops[opVertical[i][0]](
                x[0][i],
                x[1][i]
            ),
            x[2][i]
        ))
            ==
        resVertical[i]
    )

#não pode haver *1 nem /1
for i in range(3):
    for j in range(2):
        if(opHorizontal[i][j] == '/' or opHorizontal[i][j] == '*'):
            s.add(not(x[i][j+1] == 1))
        if(opVertical[i][j] == '/' or opVertical[i][j] == '*'):
            s.add(not(x[j+1][i] == 1))

#o cálculo nunca pode atingir valores negativos ou fracionários
for i in range(3):
    if(opHorizontal[i][0] == '-'):
        s.add(x[i][1] < x[i][0])
    else:
        if(opHorizontal[i][0] == '/'):
            s.add((x[i][0]%x[i][1]) == 0)

    if(opVertical[i][0] == '-'):
        s.add(x[1][i] < x[0][i])
    else:
        if(opVertical[i][0] == '/'):
            s.add((x[0][i]%x[1][i]) == 0)

Por fim, guarda-se a solução obtida.

In [ ]:
solucao = open("sol_" + nomeFicheiro, "w")

if(s.check() == sat):
    m = s.model()
    
    for i in range(6):
        if(i%2 == 0):
            solucao.write(str(m[x[int(i/2)][0]].as_long()) + opHorizontal[int(i/2)][0] + str(m[x[int(i/2)][1]].as_long()) + opHorizontal[int(i/2)][1] +str( m[x[int(i/2)][2]].as_long()) + ' ' + str(resHorizontal[int(i/2)]) + '\n')
        else:
            if(i == 5):
                solucao.write(str(resVertical[0]) + ' ' + str(resVertical[1]) + ' ' + str(resVertical[2]))
            else:
                solucao.write(opVertical[0][int(i/2)] + ' ' + opVertical[1][int(i/2)] + ' ' + opVertical[2][int(i/2)] + '\n')

else:
    solucao.write("Não tem solução.")

solucao.close()